# Combined Sensitivity Analysis

6-panel figure comparing YLL sensitivity (left column) vs GHG price sensitivity (right column).

- Row 1: Food consumption (kcal/person/day)
- Row 2: GHG emissions by food group (GtCO2eq)
- Row 3: Objective breakdown (billion USD)

In [ ]:
from pathlib import Path

import matplotlib.pyplot as plt
from sensitivity_utils import (
    aggregate_food_groups,
    assign_food_colors,
    extract_consumption_data,
    extract_ghg_data,
    extract_objective_data,
    extract_param_value,
    load_food_to_group,
    plot_objective_sensitivity,
    plot_stacked_sensitivity,
    prepare_objective_data,
)

In [ ]:
# Configuration
PROJECT_ROOT = Path("..").resolve()

# YLL config
YLL_CONFIG_NAME = "yll"
YLL_RESULTS_DIR = PROJECT_ROOT / "results" / YLL_CONFIG_NAME
YLL_PROCESSING_DIR = PROJECT_ROOT / "processing" / YLL_CONFIG_NAME

# GHG config
GHG_CONFIG_NAME = "ghg"
GHG_RESULTS_DIR = PROJECT_ROOT / "results" / GHG_CONFIG_NAME
GHG_PROCESSING_DIR = PROJECT_ROOT / "processing" / GHG_CONFIG_NAME

# Load food to group mapping
FOOD_TO_GROUP = load_food_to_group(PROJECT_ROOT)

# Constants
CONSTANT_HEALTH_VALUE_PER_YLL = 10000
CONSTANT_GHG_PRICE = 100
N_WORKERS = 8

## Load YLL Data

In [ ]:
# Find all YLL scenarios
yll_solved_dir = YLL_RESULTS_DIR / "solved"
yll_network_files = list(yll_solved_dir.glob("model_scen-yll_*.nc"))

yll_scenarios = []
for f in yll_network_files:
    scenario = f.stem.replace("model_scen-", "")
    yll_value = extract_param_value(scenario, "yll")
    if yll_value is not None:
        yll_scenarios.append((yll_value, scenario, f))

yll_scenarios.sort(key=lambda x: x[0])
print(f"Found {len(yll_scenarios)} YLL scenarios")

In [ ]:
# Extract YLL consumption data
df_yll_consumption = extract_consumption_data(
    yll_scenarios,
    YLL_PROCESSING_DIR,
    FOOD_TO_GROUP,
    YLL_RESULTS_DIR / "plots" / "yll_sensitivity.csv",
    param_name="yll_value",
    n_workers=N_WORKERS,
)

# Aggregate and prepare for plotting
df_yll_consumption_plot = aggregate_food_groups(df_yll_consumption)
min_yll = df_yll_consumption_plot.index.min()
yll_group_order = (
    df_yll_consumption_plot.loc[min_yll].sort_values(ascending=False).index.tolist()
)
df_yll_consumption_plot = df_yll_consumption_plot[yll_group_order]
yll_colors = assign_food_colors(df_yll_consumption_plot)

print(f"YLL consumption data shape: {df_yll_consumption_plot.shape}")

In [ ]:
# Extract YLL objective data
df_yll_obj = extract_objective_data(
    yll_scenarios,
    YLL_RESULTS_DIR / "plots" / "yll_objective_breakdown.csv",
    param_name="yll_value",
    constant_health_value=CONSTANT_HEALTH_VALUE_PER_YLL,
    constant_ghg_price=CONSTANT_GHG_PRICE,
    n_workers=N_WORKERS,
)
df_yll_obj = prepare_objective_data(df_yll_obj)
print(f"YLL objective data shape: {df_yll_obj.shape}")

In [ ]:
# Extract YLL GHG emissions data
df_yll_ghg = extract_ghg_data(
    yll_scenarios,
    FOOD_TO_GROUP,
    YLL_RESULTS_DIR / "plots" / "yll_ghg_by_food_group.csv",
    param_name="yll_value",
    n_workers=N_WORKERS,
)

# Aggregate and use same order as consumption plot
df_yll_ghg_plot = aggregate_food_groups(df_yll_ghg)
available_groups = [g for g in yll_group_order if g in df_yll_ghg_plot.columns]
df_yll_ghg_plot = df_yll_ghg_plot[available_groups]

print(f"YLL GHG data shape: {df_yll_ghg_plot.shape}")

## Load GHG Data

In [ ]:
# Find all GHG scenarios
ghg_solved_dir = GHG_RESULTS_DIR / "solved"
ghg_network_files = list(ghg_solved_dir.glob("model_scen-ghg_*.nc"))

ghg_scenarios = []
for f in ghg_network_files:
    scenario = f.stem.replace("model_scen-", "")
    ghg_price = extract_param_value(scenario, "ghg")
    if ghg_price is not None:
        ghg_scenarios.append((ghg_price, scenario, f))

ghg_scenarios.sort(key=lambda x: x[0])
print(f"Found {len(ghg_scenarios)} GHG scenarios")

In [ ]:
# Extract GHG consumption data
df_ghg_consumption = extract_consumption_data(
    ghg_scenarios,
    GHG_PROCESSING_DIR,
    FOOD_TO_GROUP,
    GHG_RESULTS_DIR / "plots" / "ghg_sensitivity.csv",
    param_name="ghg_price",
    n_workers=N_WORKERS,
)

# Aggregate and use same order as YLL plots (for consistent colors across columns)
df_ghg_consumption_plot = aggregate_food_groups(df_ghg_consumption)
available_groups_ghg = [
    g for g in yll_group_order if g in df_ghg_consumption_plot.columns
]
df_ghg_consumption_plot = df_ghg_consumption_plot[available_groups_ghg]

print(f"GHG consumption data shape: {df_ghg_consumption_plot.shape}")

In [ ]:
# Extract GHG objective data
df_ghg_obj = extract_objective_data(
    ghg_scenarios,
    GHG_RESULTS_DIR / "plots" / "ghg_objective_breakdown.csv",
    param_name="ghg_price",
    constant_health_value=CONSTANT_HEALTH_VALUE_PER_YLL,
    constant_ghg_price=CONSTANT_GHG_PRICE,
    n_workers=N_WORKERS,
)
df_ghg_obj = prepare_objective_data(df_ghg_obj)
print(f"GHG objective data shape: {df_ghg_obj.shape}")

In [ ]:
# Extract GHG GHG emissions data
df_ghg_ghg = extract_ghg_data(
    ghg_scenarios,
    FOOD_TO_GROUP,
    GHG_RESULTS_DIR / "plots" / "ghg_ghg_by_food_group.csv",
    param_name="ghg_price",
    n_workers=N_WORKERS,
)

# Aggregate and use same order as YLL plots (for consistent colors across columns)
df_ghg_ghg_plot = aggregate_food_groups(df_ghg_ghg)
available_groups_ghg_ghg = [g for g in yll_group_order if g in df_ghg_ghg_plot.columns]
df_ghg_ghg_plot = df_ghg_ghg_plot[available_groups_ghg_ghg]

print(f"GHG GHG data shape: {df_ghg_ghg_plot.shape}")

## Combined 6-Panel Figure

In [ ]:
# X-axis configuration
YLL_XTICKS = [1, 10, 100, 1000, 10000, 100000]
YLL_XTICKLABELS = ["0", "10", "100", "1k", "10k", "100k"]
YLL_XLABEL = "Value per Year of Life Lost [USD/YLL]"

GHG_XTICKS = [1, 10, 100, 500]
GHG_XTICKLABELS = ["0", "10", "100", "500"]
GHG_XLABEL = "GHG price [USD/tCO2eq]"

# Manual label positions for YLL plots
YLL_CONSUMPTION_LABEL_X = {
    "grain": 7,
    "dairy": 30,
    "starchy_vegetable": 5,
    "legumes": 3000,
    "oil": 5,
    "red_meat": 3,
    "sugar": 10,
    "nuts_seeds": 10000,
    "whole_grains": 1000,
    "fruits_vegetables": 300,
    "eggs_poultry": 30,
}

YLL_GHG_LABEL_X = {
    "red_meat": 3,
    "dairy": 30,
    "grain": 7,
    "oil": 10,
    "starchy_vegetable": 5,
    "legumes": 3000,
    "sugar": 5,
    "nuts_seeds": 30000,
    "whole_grains": 500,
    "fruits_vegetables": 3000,
    "eggs_poultry": 30,
}

YLL_OBJ_LABEL_X = {
    "Crop production": 50000,
    "Health burden": 10,
    "GHG cost": 3,
    "Trade": 100000,
    "Consumer values": 5000,
}

# Manual label positions for GHG plots
GHG_CONSUMPTION_LABEL_X = {
    "eggs_poultry": 8,
}

GHG_GHG_LABEL_SKIP = {"legumes", "fruits_vegetables"}

In [ ]:
# Create 3x2 multipanel figure with shared axes within rows
fig, axes = plt.subplots(3, 2, figsize=(7.08, 7.5))  # ~180mm x ~190mm

# Row 1: Food consumption
# Panel a: YLL food consumption (top-left)
plot_stacked_sensitivity(
    df_yll_consumption_plot,
    yll_colors,
    axes[0, 0],
    xlabel=YLL_XLABEL,
    ylabel="Food consumption [kcal/person/day]",
    panel_label="a",
    x_ticks=YLL_XTICKS,
    x_ticklabels=YLL_XTICKLABELS,
    label_x_positions=YLL_CONSUMPTION_LABEL_X,
    y_max=2400,
)

# Panel b: GHG food consumption (top-right) - use yll_colors for consistency
plot_stacked_sensitivity(
    df_ghg_consumption_plot,
    yll_colors,
    axes[0, 1],
    xlabel=GHG_XLABEL,
    ylabel="Food consumption [kcal/person/day]",
    panel_label="b",
    x_ticks=GHG_XTICKS,
    x_ticklabels=GHG_XTICKLABELS,
    label_x_positions=GHG_CONSUMPTION_LABEL_X,
    y_max=2400,
)

# Row 2: GHG emissions
# Panel c: YLL GHG emissions (middle-left)
plot_stacked_sensitivity(
    df_yll_ghg_plot,
    yll_colors,
    axes[1, 0],
    xlabel=YLL_XLABEL,
    ylabel="GHG emissions [GtCO2eq]",
    panel_label="c",
    x_ticks=YLL_XTICKS,
    x_ticklabels=YLL_XTICKLABELS,
    label_x_positions=YLL_GHG_LABEL_X,
    min_height_for_label=0.08,
)

# Panel d: GHG GHG emissions (middle-right) - use yll_colors for consistency
plot_stacked_sensitivity(
    df_ghg_ghg_plot,
    yll_colors,
    axes[1, 1],
    xlabel=GHG_XLABEL,
    ylabel="GHG emissions [GtCO2eq]",
    panel_label="d",
    x_ticks=GHG_XTICKS,
    x_ticklabels=GHG_XTICKLABELS,
    label_skip=GHG_GHG_LABEL_SKIP,
    min_height_for_label=0.08,
)

# Row 3: Objective breakdown
# Panel e: YLL objective breakdown (bottom-left)
plot_objective_sensitivity(
    df_yll_obj,
    axes[2, 0],
    xlabel=YLL_XLABEL,
    panel_label="e",
    x_ticks=YLL_XTICKS,
    x_ticklabels=YLL_XTICKLABELS,
    health_value=CONSTANT_HEALTH_VALUE_PER_YLL,
    ghg_price=CONSTANT_GHG_PRICE,
    label_x_positions=YLL_OBJ_LABEL_X,
    highlight_cat="GHG cost",
)

# Panel f: GHG objective breakdown (bottom-right)
plot_objective_sensitivity(
    df_ghg_obj,
    axes[2, 1],
    xlabel=GHG_XLABEL,
    panel_label="f",
    x_ticks=GHG_XTICKS,
    x_ticklabels=GHG_XTICKLABELS,
    health_value=CONSTANT_HEALTH_VALUE_PER_YLL,
    ghg_price=CONSTANT_GHG_PRICE,
    highlight_cat="Health burden",
)

# Add column titles
axes[0, 0].set_title("Health value sensitivity", fontsize=9, fontweight="bold", pad=10)
axes[0, 1].set_title("GHG price sensitivity", fontsize=9, fontweight="bold", pad=10)

# Share y-axis limits within each row
for row in range(3):
    y_min = min(axes[row, 0].get_ylim()[0], axes[row, 1].get_ylim()[0])
    y_max = max(axes[row, 0].get_ylim()[1], axes[row, 1].get_ylim()[1])
    axes[row, 0].set_ylim(y_min, y_max)
    axes[row, 1].set_ylim(y_min, y_max)

# Remove x-axis labels except for bottom row
for row in range(2):
    for col in range(2):
        axes[row, col].set_xlabel("")
        axes[row, col].set_xticklabels([])

# Remove y-axis labels from right column
for row in range(3):
    axes[row, 1].set_ylabel("")
    axes[row, 1].set_yticklabels([])

# Align y-axis labels horizontally across rows
fig.align_ylabels(axes[:, 0])

plt.tight_layout()

# Save to notebooks/figures/
output_dir = PROJECT_ROOT / "notebooks" / "figures"
output_dir.mkdir(parents=True, exist_ok=True)
output_path = output_dir / "combined_sensitivity.pdf"
plt.savefig(output_path, dpi=300, bbox_inches="tight")
print(f"Saved to: {output_path}")

plt.show()